In [50]:
import pyvista as pv
import numpy as np
from pathlib import Path
import pygmsh
import meshio

In [51]:
avatar_path = Path("../").resolve() / "data" / "avatar" / "spread_arms.obj"
avatar_mesh = pv.read(avatar_path)

avatar_mesh = avatar_mesh.scale([0.01, 0.01, 0.01])

In [52]:
z_offset = 0.15

lower_right_points = [[0, 0.95, z_offset], [0.25, 0.95, z_offset], [0.25, 1.25, z_offset]]
right_sleeve_points = [[0.35, 1.20, z_offset], [0.35, 1.45,z_offset] ]
center_points = [[0.15, 1.61,z_offset], [-0.15, 1.61,z_offset],]
left_sleeve_points = [[-0.35, 1.45,z_offset], [-0.35, 1.20, z_offset]]
lower_left_points = [[-0.25, 1.25, z_offset], [-0.25, 0.95, z_offset]]

tshirt_points = lower_right_points + right_sleeve_points + center_points + left_sleeve_points + lower_left_points
tshirt_lines = pv.lines_from_points(tshirt_points)

In [53]:
def create_mesh(closed_lines: np.ndarray) -> pv.PolyData:
    with pygmsh.geo.Geometry() as geom:
        geom.add_polygon(
            closed_lines,
            mesh_size=0.1,
        )

        mesh: meshio.Mesh = geom.generate_mesh(
            dim=2, algorithm=5, verbose=True
        )  # 5: Delaunay

    # HardCoding alert! We observe that [1] has triangle elements so we use that
    # https://github.com/meshpro/pygmsh/discussions/524
    elems = mesh.cells[1].data
    coords = mesh.points

    cells = [("triangle", elems)]
    meshio.write_points_cells("/tmp/out.vtk", coords, cells)

    mesh_pv = pv.read("/tmp/out.vtk")

    return mesh_pv

triangle_mesh_front = create_mesh(tshirt_lines.points)
# Create back mesh by copying and translating
triangle_mesh_back = triangle_mesh_front.copy(deep=True)
triangle_mesh_back.translate((0, 0, -0.35), inplace=True)

Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 20%] Meshing curve 3 (Line)
Info    : [ 30%] Meshing curve 4 (Line)
Info    : [ 40%] Meshing curve 5 (Line)
Info    : [ 50%] Meshing curve 6 (Line)
Info    : [ 60%] Meshing curve 7 (Line)
Info    : [ 70%] Meshing curve 8 (Line)
Info    : [ 80%] Meshing curve 9 (Line)
Info    : [ 90%] Meshing curve 10 (Line)
Info    : [100%] Meshing curve 11 (Line)
Info    : Done meshing 1D (Wall 0.000650391s, CPU 0.000845s)
Info    : Meshing 2D...
Info    : Meshing surface 1 (Plane, Delaunay)
Info    : Done meshing 2D (Wall 0.00244748s, CPU 0.002439s)
Info    : 101 nodes 211 elements


UnstructuredGrid,Information
N Cells,169
N Points,101
X Bounds,"-3.500e-01, 3.500e-01"
Y Bounds,"9.500e-01, 1.610e+00"
Z Bounds,"-2.000e-01, -2.000e-01"
N Arrays,0


In [54]:
center = (0, 1.1, 0.25)

plotter = pv.Plotter()
plotter.add_mesh(avatar_mesh, color="yellow")
plotter.add_mesh(triangle_mesh_back, color = "blue", show_edges=True)
plotter.add_mesh(triangle_mesh_front, color = "blue", show_edges=True)

edge_points = triangle_mesh_front.extract_feature_edges(boundary_edges=True, 
                                      non_manifold_edges=False, 
                                      manifold_edges=False)

# plotter.add_point_labels(edge_points.points, labels = [str(point) for point in edge_points.points])

plotter.add_axes()
plotter.show(cpos = 'xy')

ViewInteractiveWidget(height=768, layout=Layout(height='auto', width='100%'), width=1024)

In [55]:
edge_point_indexes = [triangle_mesh_front.find_closest_point(point) for point in edge_points.points] 

print(edge_point_indexes)
for index in edge_point_indexes[5:]:
    print(index, triangle_mesh_front.points[index])

[25, 8, 26, 15, 3, 16, 22, 23, 5, 20, 0, 11, 18, 19, 17, 4, 7, 24, 21, 30, 12, 28, 10, 1, 13, 6, 14, 27, 9, 2, 29]
16 [0.35       1.28333333 0.15      ]
22 [-0.21666667  1.55666667  0.15      ]
23 [-0.28333333  1.50333333  0.15      ]
5 [0.15 1.61 0.15]
20 [0.05 1.61 0.15]
0 [0.   0.95 0.15]
11 [0.08333333 0.95       0.15      ]
18 [0.28333333 1.50333333 0.15      ]
19 [0.21666667 1.55666667 0.15      ]
17 [0.35       1.36666667 0.15      ]
4 [0.35 1.45 0.15]
7 [-0.35  1.45  0.15]
24 [-0.35        1.36666667  0.15      ]
21 [-0.05  1.61  0.15]
30 [-0.08333333  0.95        0.15      ]
12 [0.16666667 0.95       0.15      ]
28 [-0.25  1.05  0.15]
10 [-0.25  0.95  0.15]
1 [0.25 0.95 0.15]
13 [0.25 1.05 0.15]
6 [-0.15  1.61  0.15]
14 [0.25 1.15 0.15]
27 [-0.25  1.15  0.15]
9 [-0.25  1.25  0.15]
2 [0.25 1.25 0.15]
29 [-0.16666667  0.95        0.15      ]


In [56]:
triangle_mesh_front_file = Path("../").resolve() / "data" / "avatar" / "tshirt_front.obj"
triangle_mesh_back_file = Path("../").resolve() / "data" / "avatar" / "tshirt_back.obj"
pv.save_meshio(filename=triangle_mesh_front_file, mesh=triangle_mesh_front)
pv.save_meshio(filename=triangle_mesh_back_file, mesh=triangle_mesh_back)

In [57]:

# Define a simple Gaussian surface
n = 20
x = np.linspace(-200, 200, num=n) + np.random.uniform(-5, 5, size=n)
y = np.linspace(-200, 200, num=n) + np.random.uniform(-5, 5, size=n)
xx, yy = np.meshgrid(x, y)
A, b = 100, 100
zz = A * np.exp(-0.5 * ((xx / b) ** 2.0 + (yy / b) ** 2.0))

# Get the points as a 2D NumPy array (N by 3)
points = np.c_[xx.reshape(-1), yy.reshape(-1), zz.reshape(-1)]
points[0:5, :]

array([[-200.83954945, -202.52254756,    1.71185032],
       [-179.22400319, -202.52254756,    2.58141812],
       [-153.32116671, -202.52254756,    3.97105604],
       [-139.0765832 , -202.52254756,    4.89045953],
       [-119.88944092, -202.52254756,    6.26969162]])